## MTH377 - Convex Optimisation (COO) - Assessment 1

### Problem 2

- We have been given the function f(x,y):
$$
f(x,y) = (x^2 - 3y^2)^2 + \sin^2(x^2+y^2)
$$

- We need to minimise this function as an unconstrained optimisation problem using the combination descent algorithm with starting point at (1,1).

In [1]:
import numpy as np

# Define the function
def f(x,y):
    return (np.sin(x**2 + y**2))**2 + (x**2 - 3*(y**2))**2

- For the gradient, we need expressions for the partial derivates of f(x,y) wrt x and y:
$$
\frac{\partial f}{\partial x} = 4x(x^2-3y^2) + 2x\sin(2(x^2 + y^2))

$$
$$
\frac{\partial f}{\partial y} = -12y(x^2-3y^2) + 2y\sin(2(x^2 + y^2))
$$

- Thus the gradient of f(x,y) is a vector of the form:

$$
\nabla f = \begin{vmatrix} 
\frac{\partial f}{\partial x} \\
\\
\frac{\partial f}{\partial y}
\end{vmatrix}
$$

In [2]:
# Define the gradient
def gradient_f(x,y):

    # Implementing the partial derivatives as derived above
    df_dx = 4*x*(x**2 - 3*(y**2)) + 4*(np.sin(x**2 + y**2))*(np.cos(x**2 + y**2))*x
    df_dy = (-12)*y*(x**2 - 3*(y**2)) + 4*(np.sin(x**2 + y**2))*(np.cos(x**2 + y**2))*y

    # Returning the gradient as a numpy array of the partial derivatives
    return np.array([df_dx, df_dy])

- For the Hessian Matrix, we need the double partial derivates with respect to x and y. Calculated them as follows:
$$ \frac{\partial^{2} f}{\partial x^2} = 12x^2 - 12y^2 +2\sin(2(x^2 + y^2)) + 8x^2\cos(2(x^2 + y^2)) $$
$$ \frac{\partial^{2} f}{\partial xy} = \frac{\partial^{2} f}{\partial yx} = -24xy + 8xy\cos(2(x^2 + y^2)) $$
$$ \frac{\partial^{2} f}{\partial y^2} = 108y^2 - 12x^2 +2\sin(2(x^2 + y^2)) + 8y^2\cos(2(x^2 + y^2)) $$

- Thus the Hessian Matrix of the given function f(x,y) at any (x,y) will be:
$$
H(x,y) = \begin{vmatrix} 
\frac{\partial^{2} f}{\partial x^2}    \frac{\partial^{2} f}{\partial xy} \\
\\
\frac{\partial^{2} f}{\partial yx}     \frac{\partial^{2} f}{\partial y^2} 
\end{vmatrix}

In [3]:
# Define the Hessian
def hessian_f(x,y):

    # Implementing the expressions as derived above
    df2_dxdx = 12*(x**2) - 12*(y**2) +2*(np.sin(2*(x**2 + y**2))) + (8*(x**2))*np.cos(2*(x**2 + y**2))
    df2_dxdy = -24*x*y + (8*x*y)*np.cos(2*(x**2 + y**2))
    df2_dydy = 108*(y**2) - 12*(x**2) +2*(np.sin(2*(x**2 + y**2))) + (8*(y**2))*np.cos(2*(x**2 + y**2))

    # Returning the Hessian as a 2x2 matrix
    return np.array([[df2_dxdx, df2_dxdy], [df2_dxdy, df2_dydy]])

- We find the descent direction 'd' using the Backtracking-Linear Search Routine or the Armijo Rule.
- We check for the following inequality to be true:
$$
f(x) - f(x+td) < \alpha(-Df(x)·(td))
$$
where $\alpha$ is an acceptable descent parameter $\in (0,0.5)$

- We start with t = 1 and then till the about condition holds true, we multiply t with a reduction factor $\beta$ $\in (0,1)$

- I have implemented the same below. In our case $d$ would be a 1x2 matrix and $Df(x)$ becomes the gradient $\nabla f$ at the original point $x,y$.

In [4]:
# Defining the Backtracking Line Search
def linear_search(d, alpha, beta, point):
    t = 1
    while((f(point[0], point[1]) - f(point[0] + t*d[0], point[1] + t*d[1])) < (-alpha*t*np.dot(gradient_f(point[0], point[1]), d))): 
        t = beta*t
    return t


- We implement the Combination Descent Algorithm below.
- First, we have a function `is_pos_def` which checks if a matrix is positive definite by seeing if all its eigenvalues are positive. 
- This works for a symmetric matrix and we can do this test on the Hessian Matrix $Hf(x)$ because the Hessian Matrix is always symmetric as $\frac{\partial^{2} f}{\partial xy} = \frac{\partial^{2} f}{\partial yx}$ (in the context of $R^2$)

- If our matrix is positive definite, we choose our $d = (Hf(x))^{-1}(-\nabla f(x))$ or the $d = -\nabla f(x)$
- Then we apply the Linear Search method to the step-size $t$ which we then use with the desent direction $d$ to update our x and y coordinates.
- We perform this in a loop till the value of $\nabla f(x) \leq \eta $ where $\eta$ is the tolerance level set by us.

In [5]:
# Function to check if a matrix is positive definite
def is_pos_def(x):

    # For checking if a matrix is positive definite we check if all the eigenvalues are positive (applicable for symmetric matrices)
    return np.all(np.linalg.eigvals(x) > 0)


# Implementing the Combination Descent Method
def combination_descent(x0, y0, alpha, beta, epsilon, max_iters = 100):
    # Initialize the variables
    x = x0
    y = y0

    #Initializing the loop
    while (np.linalg.norm(gradient_f(x,y)) > epsilon) and (max_iters != 0):
        
        # Compute the gradient at the current point (x,y)
        grad = gradient_f(x,y)

        # Compute the Hessian at the current point (x,y)
        hess = hessian_f(x,y)

        # Combination Descent Condition:
        if(is_pos_def(hess)):

            # If the Hessian is positive definite, we use the Newton Step
            
            # Computing the inverse of the Hessian
            hess_inverse = np.linalg.inv(hess)

            # Computing the Newton Step by taking the negative of the Hessian inverse dotted with the gradient vector
            d = -np.dot(hess_inverse, grad)
        
        else:

            # If the Hessian is not positive definite, we use the Gradient Descent Step which is just the negative of the gradient vector
            d = -grad

        # Compute the step size
        t = linear_search(d, alpha, beta, [x,y])

        # Update the variables
        x = x + t*d[0]
        y = y + t*d[1]

        # Update the number of iterations
        max_iters = max_iters - 1

    # Return the solution as a numpy array with the number of iterations it took and the coordinates of the solution point
    return np.array([x, y, max_iters])

- Here we define the following values:
$$
(x_0,y_0) = (1,1)
$$
$$
\alpha = 0.1
$$
$$
\beta = 0.3
$$
$$
\eta = 10^{-7}
$$
- We then start the combination descent algorithm

In [6]:
# Define the initial point
x0 = 1
y0 = 1

# Defining Constants
alpha = 0.2
beta = 0.5
epsilon = 1e-6

# Calling the function
point = combination_descent(x0, y0, alpha, beta, epsilon)

# Printing the results
print("The point of minima for the given function is at: (%.4f , %.4f)" %(point[0], point[1]))
print("The approximate minimum value of the given function is: %.4f" %(f(point[0], point[1])))

The point of minima for the given function is at: (1.5350 , 0.8862)
The approximate minimum value of the given function is: 0.0000


- We get the minima approximately for $f(x,y)$ at $(0.0021, 0.0009)$.
- The approximate minimum value of the function hence is $0.00$